In [17]:
import PIL
import os
import imageio
import numpy as np
from six.moves import cPickle as pickle
import tensorflow as tf

In [18]:
#Script handles packaging data from folder into folder.pickle dataset
#Checks as well if images going into dataset are of proper size and adjusts their color depth
image_size = 256
pixel_depth = 255.0
channels = 3
ratio = 0.8
src_dir = 'Train'

def load_picture(folder):
    image_files = os.listdir(folder)
    dataset = np.ndarray(shape=(len(image_files),image_size,image_size,channels),dtype=np.float32)
    print(dataset.shape)
    num_images = 0
    for image in image_files:
        image_file = os.path.join(folder, image)
        name = (os.path.splitext(image))
        num_extracted = int(name[0])
        try:
            image_data = (imageio.imread(image_file).astype(float) - pixel_depth/2)/pixel_depth
            if image_data.shape != (image_size,image_size,channels):
                raise Exception('Wrong image shape {}'.format(image_file))
            dataset[num_extracted-1,:,:,:] = image_data
            #print(num_images)
        except(IOError, ValueError) as e:
            print("Could not read:", image_file,":",e)
    return dataset

In [19]:
def do_pickle(folder, force=False):
    set_filename = folder + '.pickle'
    if os.path.exists(set_filename) and not force:
        print("You already have dataset present")
    else:
        print("Pickling {}".format(set_filename))
        dataset = load_picture(folder)
        try:
            with open(set_filename, 'wb') as f:
                pickle.dump(dataset,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print("Unable to save data",e)
    print("Done")
    return True

In [20]:
train_datasets = do_pickle(src_dir, force = True)

Pickling Train.pickle
(100, 256, 256, 3)
Done


In [6]:
pickle_file = "Train.pickle"

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
  print('completed')
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

completed
